In [5]:
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import nltk
import pickle
import re
import string

from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import punkt
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import wordnet
from nltk.corpus import stopwords

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

from exclude_remove import bracket_exclusions, reg_excludes

print('done')


done


In [6]:
data_path = '../data/processed/'
# read in CPA data
CPA = pd.read_csv(data_path + "CPA_2_1.csv", skiprows=1, sep=',',                  
                  names=['Order','Level','Code','Parent','Description','Includes','Also','Rulings','Excludes'])
# read in SITC data
SITC = pd.read_csv(data_path+'SITC_Rev3_UN.csv', skiprows=2, names=['Code','Description'])
# Read in CN data file
CN = pd.read_csv(data_path+"CN_2020.csv", header=None, sep=',', names=['Code','Section','Description'])

# read in other codes data
BEC = pd.read_csv(data_path + "BEC_Rev_5.csv", skiprows=1, names=['Code','BEC'], index_col=False)
COICOP = pd.read_csv(data_path+'COICOP_2018.csv')
#PRD = pd.read_csv(data_path+ "PRD_2019.csv")

# read in mapping data
#CN_CPA = pd.read_csv(data_path+'CN 2020-CPA 2.1.csv', skiprows=1)
#CPA_COI = pd.read_csv(data_path + 'COICOP 2018 - CPA 2 1.csv')
SITC
CPA.head()

,Order,Level,Code,Parent,Description,Includes,Also,Rulings,Excludes
0,1208792,1,A,NaN,"PRODUCTS OF AGRICULTURE, FORESTRY AND FISHING",NaN,NaN,NaN,NaN
1,1208793,2,01,A,"Products of agriculture, hunting and related s...",NaN,NaN,NaN,NaN
2,1208794,3,01.1,01,Non-perennial crops,NaN,NaN,NaN,NaN
3,1208795,4,01.11,01.1,"Cereals (except rice), leguminous crops and oi...","This class includes:\n- cereals, leguminous cr...",NaN,NaN,NaN
4,1208796,5,01.11.1,01.11,Wheat,NaN,NaN,NaN,NaN


In [7]:
# function to print all content in one column as a string
def get_string(col1):
    col = col1[col1.notnull()]
    print(len(col))
    for i in range(len(col)):
        tmp_str = col.iloc[i]
        if tmp_str != '':
            print(tmp_str, '\n') #col.iloc[i],'\n')
# print out all content from two columns as pairs of strings        
def get_string2(cols):
    print(len(cols))
    for i in range(len(cols.iloc[:,0])):
        print(cols.iloc[i,0],'\n', cols.iloc[i,1],'\n')
        
def search_col(df, col, st):
    st1 = st.lower()
    df1 = df[df[col].fillna('').str.contains(st1)]
    get_string(df1[col])

# basic regex cleaning
def reg1(df):
    df = df.replace(r'\n-','',regex=True)
    df = df.replace(r'\n','',regex=True)
    df = df.replace(r'\t','',regex=True)
    df = df.replace(r'\r -','',regex=True)
    return df

COICOP = reg1(COICOP)
BEC = reg1(BEC)
SITC = reg1(SITC)
#CPA_COI = reg1(CPA_COI)
CPA = reg1(CPA)
CN  = reg1(CN)

# BEC.head(20)
# COICOP.head(20)
# CPA.head()

## Cleaning CPA data

In [8]:
# first we combine columns with "includes" information
CPA1 = CPA.copy()
CPA1['Includes'] = CPA1['Includes'].fillna('')  + CPA1['Also'].fillna('') + CPA1['Rulings'].fillna('')
CPA1['Includes_cleaned'] = CPA1.Includes.str.replace('This(.*)includes: |This(.*)includes ','') 


CPA1 = CPA1.drop(['Rulings','Excludes','Also','Includes'], axis=1)



In [9]:
# function to remove phrases such as "except...", "excluding..." that appear at the end of the description
# function to remove exclusions from the description which appear in brackets
# def bracket_exclusions(df1,col_old):
#     df= df1.copy()
#     ex_str = r'(\(except(.*?)\)|\(excl(.*?)\)|\(without(.*?)\)|\(not (.*?)\)|\(other than (.*?)\))'
#     df['Excl_removed'] = ''
#     df['Excl_removed'] = df[col_old].str.extract(ex_str)
#     df[col_old+'_excl_rem'] = df[col_old].str.replace(ex_str, '')
#     return df

# def reg_excludes(df1,col_old, excl_col):
#     df= df1.copy()
#     df['Temp'] = df[col_old]
#     # remove exceptions in brackets
#     ex_str = r'(\(except(.*?)\)|\(excl(.*?)\)|\(without(.*?)\)|\(not (.*?)\)|\(other than (.*?)\))'
#     # unnecessary phrases:
#     ex_str0 = r'(This(.*)excludes: |This(.*)excludes |This(.*)includes: |This(.*)includes )'
#     # sometimes there is "include" info after a phrase of the pattern "except... ;" or "excelpt... ."
#     ex_str1 = r'(, except.*?\.|, excl.*?\.|, without.*\.|, not\b.*?\.| not\b.*?\.| not\b.*?\)|, other than .*?\.)'
#     ex_str1a = r'(, except.*?;|, excl.*?;|, without.*;|,\bnot\b.*?;|\bnot\b.*?;| not\b.*?\)|, other than .*?;)'

#     # sometimes there is "include" info after a phrase of the pattern  "except..., and"
#     ex_str1b = r'(, except.*?(, and)|, excl.*?(, and)|, without.*(, and))'
#     # the phrases separated by a comma, eg "except..., " usually are followed by "exclude" info and occur at the end of the description.
#     # however, phrases such as "not...", "other than..." which may have more "include" data later in the Description that must be left in.
#     ex_str1c = r'(, whether or not.*?,)'
#     # we now assume that the "except..." phrases continue to the end of the description field.
#     ex_str2 = '(, except.*|, excl(.*)|, without .*| not\s.*|, other than .*)'    
#     # other things to exclude
#     ex_str3 = r'(not elsewhere classified | n.e.c. | n.e.c)'

#     excl_list = []

#     i = 1
#     ex_strs = [ex_str, ex_str0, ex_str1, ex_str1a, ex_str1b, ex_str1c, ex_str2, ex_str3]
#     for s in ex_strs:
#         name = f'{excl_col}_{str(i)}'
#         print(name)
#         df[name] = ''
#         df[name]= df['Temp'].str.extract(s, expand=True)
#         df['Temp'] = df['Temp'].str.replace(s, '')
        
#         df[excl_col] = df[excl_col].fillna('') + df[name].fillna('') 
#         excl_list.append(name)

#         i+=1
#       #  get_string(df[df[name].notnull()][name])
#     # create our new column with these items excluded
#     df[col_old+'_excl_rem'] = df['Temp']
#     # prepare dataframe to be returned

#     df2 = df.copy().drop(excl_list +['Temp',col_old],axis=1)

#     return df2

CPA2 = CPA1.rename(columns={'Description':'Descr_old'})
CPA2['Descr'] = CPA2['Descr_old'].copy()
CPA2['Excl_removed'] = ''
CPA3 = reg_excludes(CPA2,'Descr','Excl_removed')
CPA3['Inc_removed'] = ''
CPA4 = reg_excludes(CPA3,'Includes_cleaned','Inc_removed')

CPA3.sample(3)
#get_string(CPA3[CPA3.Inc_removed!='']['Inc_removed'])
#get_string(CPA3[CPA3.Excl_removed_7.notnull()]['Excl_removed'])
CPA3

Excl_removed_1
Excl_removed_2
Excl_removed_3
Excl_removed_4
Excl_removed_5
Excl_removed_6
Excl_removed_7
Excl_removed_8
Inc_removed_1
Inc_removed_2
Inc_removed_3
Inc_removed_4
Inc_removed_5
Inc_removed_6
Inc_removed_7
Inc_removed_8


,Order,Level,Code,Parent,Descr_old,Includes_cleaned,Excl_removed,Descr_excl_rem,Inc_removed
0,1208792,1,A,NaN,"PRODUCTS OF AGRICULTURE, FORESTRY AND FISHING",,,"PRODUCTS OF AGRICULTURE, FORESTRY AND FISHING",
1,1208793,2,01,A,"Products of agriculture, hunting and related s...",,,"Products of agriculture, hunting and related s...",
2,1208794,3,01.1,01,Non-perennial crops,,,Non-perennial crops,
3,1208795,4,01.11,01.1,"Cereals (except rice), leguminous crops and oi...","cereals, leguminous crops and oil seeds grown ...",(except rice),"Cereals , leguminous crops and oil seeds",
4,1208796,5,01.11.1,01.11,Wheat,,,Wheat,
...,...,...,...,...,...,...,...,...,...
5517,1214309,2,99,U,Services provided by extraterritorial organisa...,,,Services provided by extraterritorial organisa...,
5518,1214310,3,99.0,99,Services provided by extraterritorial organisa...,,,Services provided by extraterritorial organisa...,
5519,1214311,4,99.00,99.0,Services provided by extraterritorial organisa...,,,Services provided by extraterritorial organisa...,
5520,1214312,5,99.00.1,99.00,Services provided by extraterritorial organisa...,,,Services provided by extraterritorial organisa...,


In [10]:
CPA_cl = CPA3[['Order','Level','Code','Parent','Descr_old','Descr_excl_rem','Includes_cleaned']].rename(columns = {'Descr_excl_rem':'Descr','Includes_cleaned':'Includes'}).copy()
CPA_cl.to_csv('../data/processed/CPA_data_cleaned.csv', index=False)

In [141]:

CPA_L6 = CPA2[CPA2.Level==6].copy()

# join the L5 description to the L6
CPA_L5 = CPA2[CPA2.Level==5].copy()

CPA_L56 = CPA_L6.merge(CPA_L5[['Code','Descr']].rename(columns={'Descr':'L5_Descr'}), left_on='Parent', right_on='Code', how='left')
CPA_L56 = CPA_L56.drop(['Code_x','Code_y','Parent'], axis=1)
CPA_L56.sample(5)

,Order,Level,Descr,Includes_cleaned,Excl_removed,L5_Descr
1850,1211849,6,Parts and accessories of bicycles and other cy...,,,Parts and accessories of bicycles and other cy...
3099,1214063,6,Pregnancy related services,"services provided by authorised persons, other...",,Other human health services
3174,1214220,6,Services provided by cultural and recreational...,services provided by cultural or recreational ...,,Services (except grant-giving services) furnis...
3129,1214120,6,Services of performing artists,"services of actors, readers, singers, musician...",,Services of performing artists
1548,1211366,6,Parts of steam and other vapour turbines,,,Parts of turbines


## Other data